In [1]:
 
import pandas as pd
df = pd.read_csv("dataset.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,male,cardiac,145,233,sugar,normal,150,no,2.3,up,0,Normal,1
1,37,male,abnormal,130,250,no_sugar,abnormal,187,no,3.5,up,0,Fixed defect,1
2,41,female,asymptomatic,130,204,no_sugar,normal,172,no,1.4,down,0,Fixed defect,1
3,56,male,asymptomatic,120,236,no_sugar,abnormal,178,no,0.8,down,0,Fixed defect,1
4,57,female,angina,120,354,no_sugar,abnormal,163,yes,0.6,down,0,Fixed defect,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,female,angina,140,241,no_sugar,abnormal,123,yes,0.2,flat,0,Reversial defect,0
299,45,male,cardiac,110,264,no_sugar,abnormal,132,no,1.2,flat,0,Reversial defect,0
300,68,male,angina,144,193,sugar,abnormal,141,no,3.4,flat,2,Reversial defect,0
301,57,male,angina,130,131,no_sugar,abnormal,115,yes,1.2,flat,1,Reversial defect,0


In [5]:
df.max()

age                        77
sex                      male
cp                    cardiac
trestbps                  200
chol                      564
fbs                     sugar
restecg                normal
thalach                   202
exang                     yes
oldpeak                   6.2
slope                      up
ca                          4
thal        Reversial  defect
target                      1
dtype: object

In [6]:
X = df.drop(columns='target', axis=1)
Y = df['target']

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
num_column = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak','ca']
cat_column = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope','thal']
cat_pipeline = Pipeline([("one hot encoder",OneHotEncoder(handle_unknown="ignore"))])
num_pipeline = Pipeline([("scaler",StandardScaler())])
transformer = ColumnTransformer([("Categorical",cat_pipeline,cat_column),("Numerical",num_pipeline,num_column)])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, stratify=Y, random_state=2)

In [11]:
model = Pipeline([("Transforming",transformer),("xgb",XGBClassifier())])
model.fit(X_train,Y_train)

Pipeline(steps=[('Transforming',
                 ColumnTransformer(transformers=[('Categorical',
                                                  Pipeline(steps=[('one hot '
                                                                   'encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'cp', 'fbs',
                                                   'restecg', 'exang', 'slope',
                                                   'thal']),
                                                 ('Numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'trestbps', 'chol',
                                                   'thalach', 'oldpeak',
                                                   'ca'])])),
                ('xgb',
                 XGBClassifier(base_s...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [12]:
y_pred = model.predict(X_test)

In [13]:
result = model.score(X_test, Y_test)
result*100

81.57894736842105

In [15]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
training_data_accuracy

1.0

In [16]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
test_data_accuracy

0.8157894736842105

In [18]:
import joblib
joblib.dump(model,'heart.joblib')

['heart.joblib']